# Data Ingestion P2: with MySQL

In [1]:
# Import libraries for data preparation & transformation
import numpy as np 
import pandas as pd 
import os 
import sys

# Import mysql for building connectivity <mysql, python>
import mysql.connector as conn 

In [2]:
# System configuration
sys.path.append(os.path.abspath(".."))

## Approach

+ Build a connection between mysql and python through python package
+ Extract all avalable data within the lpp_raw_db database: numerical_properties, categorical_properties and complex_properties
+ Split tasks into sub tasks for each dataset

## MySQL Database Connectivity with Python

In [3]:
# Make connection with MySQL database
cnx = conn.connect(
    user="root",
    password="@London98",
    host="127.0.0.1",
    database="laptop_raw_db"
)

# Create cursor for sql-querying
def query_sql_tables(query: str):
    # Establish connection with MySQL
    cnx = conn.connect(
    user="root",
    password="@London98",
    host="127.0.0.1",
    database="laptop_raw_db"
)
    # Create cursor to execute query
    cursor = cnx.cursor()
    cursor.execute(query)

    # Extract relevant parameters for building a dataset
    column_names = cursor.column_names
    data_samples = list(cursor)
    dataset = pd.DataFrame(data_samples, columns=column_names)

    cnx.close()

    return dataset

## Data Collection from MySQL

In [4]:
# Querying: Extract 3 tables from mysql
query1 = "SELECT * FROM CATEGORICAL_PROPERTIES"
query2 = "SELECT * FROM NUMERICAL_PROPERTIES"

dataset1 = query_sql_tables(query1) # Categorical Dataset
dataset2 = query_sql_tables(query2) # Numerical Dataset

In [5]:
dataset1 

,LAPTOP_ID,BRAND,CURRENCY,COLOR,CONDITIONS,GPU,PROCESSOR,PROCESSOR_SPEED_UNIT,LAPTOP_TYPE,OPERATING_SYSTEM,STORAGE_TYPE,HARD_DRIVE_CAPACITY_UNIT,SSD_CAPACITY_UNIT,RAM_SIZE_UNIT
0,1,other,$,gray,New,intel,quad core,GHz,notebook/laptop,windows,ssd,gb,tb,gb
1,2,dell,$,black,Used,intel,intel core i5-6300u,GHz,notebook/laptop,windows,ssd,gb,gb,gb
2,3,hp,$,black,Good - Refurbished,intel,intel celeron n,GHz,notebook/laptop,chrome,emmc,gb,unknown,gb
3,4,dell,$,other,Good - Refurbished,other,intel core i5 6th generation,GHz,notebook/laptop,windows,ssd,unknown,gb,gb
4,5,acer,$,black,Undefined,other,intel celeron,GHz,notebook/laptop,chrome,ssd,unknown,gb,gb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4177,4178,other,$,other,New,other,undefined,unknown,other,unknown,unknown,unknown,unknown,unknown
4178,4179,acer,$,other,New,other,not applicable,unknown,other,unknown,unknown,unknown,unknown,unknown
4179,4180,dell,$,black,Used,intel,intel core i5 6th generation,GHz,notebook/laptop,windows,unknown,unknown,gb,unknown
4180,4181,hp,$,other,New,other,not applicable,unknown,other,unknown,unknown,unknown,unknown,unknown


In [6]:
dataset2

,LAPTOP_ID,PRICE,WIDTH_DISPLAY,HEIGHT_DISPLAY,HARD_DRIVE,SSD_CAPACITY,RAM_SIZE,PROCESSOR_SPEED,SCREEN_SIZE_INCH
0,1,303.80,2160,1440,512,1,8,3.8,14.00
1,2,175.00,1920,1080,500,500,16,2.4,14.00
2,3,85.00,1366,768,16,240,4,2.4,11.60
3,4,101.25,1366,768,256,256,8,1.4,12.50
4,5,50.00,1366,768,256,16,4,1.6,11.60
...,...,...,...,...,...,...,...,...,...
4177,4178,162.20,1920,1080,256,240,8,0.0,8.43
4178,4179,93.25,1920,1080,256,240,8,0.0,8.43
4179,4180,424.80,1920,1080,256,120,8,2.8,12.50
4180,4181,90.94,1920,1080,256,240,8,0.0,8.43


## Traformation Process

Starting from raw dataset and transform to 4 main external dataset: 

- visual_property_data.csv -> **visual_dataset**
- network_property_data.csv -> **network_dataset**
- hardware_property_data.csv -> **hardware_dataset**
- descriptive_property_data.csv -> **descriptive_dataset**

Approach for this process:
- Write down lists of features allocating to the correct desired dataset
- Store the dataset into the **processed** folder

In [7]:
dataset1.columns

Index(['LAPTOP_ID', 'BRAND', 'CURRENCY', 'COLOR', 'CONDITIONS', 'GPU',
       'PROCESSOR', 'PROCESSOR_SPEED_UNIT', 'LAPTOP_TYPE', 'OPERATING_SYSTEM',
       'STORAGE_TYPE', 'HARD_DRIVE_CAPACITY_UNIT', 'SSD_CAPACITY_UNIT',
       'RAM_SIZE_UNIT'],
      dtype='object')

In [8]:
dataset2.columns

Index(['LAPTOP_ID', 'PRICE', 'WIDTH_DISPLAY', 'HEIGHT_DISPLAY', 'HARD_DRIVE',
       'SSD_CAPACITY', 'RAM_SIZE', 'PROCESSOR_SPEED', 'SCREEN_SIZE_INCH'],
      dtype='object')

## Data Creation

In [9]:
# Import functionalities to build mysql tables 
from scripts.processing.mysql_manager import MySQL_ID_Creator

In [10]:
# 4 external datasets
visual_data = pd.DataFrame()
memory_data = pd.DataFrame()
network_data = pd.DataFrame()
hardware_data = pd.DataFrame()
information_data = pd.DataFrame()

# 4 external feature data
visual_features = ["SCREEN_SIZE_INCH", "COLOR", "WIDTH_DISPLAY", "HEIGHT_DISPLAY"]
hardware_features = ["GPU", "RAM_SIZE_UNIT","STORAGE_TYPE", "OPERATING_SYSTEM"]
processor_features = ["PROCESSOR" ,"PROCESSOR_SPEED","PROCESSOR_SPEED_UNIT"]
memory_features = ["HARD_DRIVE", "HARD_DRIVE_CAPACITY_UNIT", "SSD_CAPACITY", "SSD_CAPACITY_UNIT"]
information_features = ['BRAND', 'CURRENCY', 'COLOR', 'CONDITIONS', "PRICE"]
network_features = []

# List of ID features
id_features = ["SCREEN_ID","HARDWARE_ID","PROCESSOR_ID","MEMORY_ID","INFO_ID","NETWORK_ID"]

In [11]:
# Instantiate MySQL_table creator object
sql_creator = MySQL_ID_Creator(full_dataset=dataset1)

# Build screen id
screen_id = sql_creator.build_id_feature("SCREEN_ID")
memory_id = sql_creator.build_id_feature("MEMORY_ID")
processor_id = sql_creator.build_id_feature("PROCESSOR_ID")
info_id = sql_creator.build_id_feature("INFO_ID")
hardware_id = sql_creator.build_id_feature("HARDWARE_ID")
network_id = sql_creator.build_id_feature("NETWORK_ID")

id_features = ["SCREEN_ID","HARDWARE_ID","PROCESSOR_ID","MEMORY_ID","INFO_ID","NETWORK_ID"]
feature_id = [screen_id, hardware_id, processor_id, memory_id, info_id, network_id]

# Show all available primary & foreign id features
data_id_gr = {}
for ids, feature in zip(feature_id, id_features):
    print(f"{feature}: {ids[:10]}")

    # Convert into dataframe
    feature_id_name = f"{feature}_df"
    data = pd.DataFrame()
    data[feature] = sql_creator.create(feature_id_name)
    data_id_gr[feature_id_name] = data[feature]

SCREEN_ID: ['S0' 'S1' 'S2' 'S3' 'S4' 'S5' 'S6' 'S7' 'S8' 'S9']
HARDWARE_ID: ['H0' 'H1' 'H2' 'H3' 'H4' 'H5' 'H6' 'H7' 'H8' 'H9']
PROCESSOR_ID: ['P0' 'P1' 'P2' 'P3' 'P4' 'P5' 'P6' 'P7' 'P8' 'P9']
MEMORY_ID: ['M0' 'M1' 'M2' 'M3' 'M4' 'M5' 'M6' 'M7' 'M8' 'M9']
INFO_ID: ['I0' 'I1' 'I2' 'I3' 'I4' 'I5' 'I6' 'I7' 'I8' 'I9']
NETWORK_ID: ['N0' 'N1' 'N2' 'N3' 'N4' 'N5' 'N6' 'N7' 'N8' 'N9']


## Data Transformation

In [12]:
# Import functionalities to combine data 
from scripts.processing.transformer import ColumnTransformer
from scripts.processing.mysql_manager import MySQL_Table_Creator

In [13]:
# Combine both datasets: dataset 1 and 2
combiner = ColumnTransformer(dataset1)
dataset = combiner.combine(dataset2)
dataset

,LAPTOP_ID,BRAND,CURRENCY,COLOR,CONDITIONS,GPU,PROCESSOR,PROCESSOR_SPEED_UNIT,LAPTOP_TYPE,OPERATING_SYSTEM,...,RAM_SIZE_UNIT,LAPTOP_ID,PRICE,WIDTH_DISPLAY,HEIGHT_DISPLAY,HARD_DRIVE,SSD_CAPACITY,RAM_SIZE,PROCESSOR_SPEED,SCREEN_SIZE_INCH
0,1,other,$,gray,New,intel,quad core,GHz,notebook/laptop,windows,...,gb,1,303.80,2160,1440,512,1,8,3.8,14.00
1,2,dell,$,black,Used,intel,intel core i5-6300u,GHz,notebook/laptop,windows,...,gb,2,175.00,1920,1080,500,500,16,2.4,14.00
2,3,hp,$,black,Good - Refurbished,intel,intel celeron n,GHz,notebook/laptop,chrome,...,gb,3,85.00,1366,768,16,240,4,2.4,11.60
3,4,dell,$,other,Good - Refurbished,other,intel core i5 6th generation,GHz,notebook/laptop,windows,...,gb,4,101.25,1366,768,256,256,8,1.4,12.50
4,5,acer,$,black,Undefined,other,intel celeron,GHz,notebook/laptop,chrome,...,gb,5,50.00,1366,768,256,16,4,1.6,11.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4177,4178,other,$,other,New,other,undefined,unknown,other,unknown,...,unknown,4178,162.20,1920,1080,256,240,8,0.0,8.43
4178,4179,acer,$,other,New,other,not applicable,unknown,other,unknown,...,unknown,4179,93.25,1920,1080,256,240,8,0.0,8.43
4179,4180,dell,$,black,Used,intel,intel core i5 6th generation,GHz,notebook/laptop,windows,...,unknown,4180,424.80,1920,1080,256,120,8,2.8,12.50
4180,4181,hp,$,other,New,other,not applicable,unknown,other,unknown,...,unknown,4181,90.94,1920,1080,256,240,8,0.0,8.43


In [14]:
# Build primary & foreing id feature data
screen_id_df = data_id_gr["SCREEN_ID_df"]
hardware_id_df = data_id_gr["HARDWARE_ID_df"]
processor_id_df = data_id_gr["PROCESSOR_ID_df"]
memory_id_df = data_id_gr["MEMORY_ID_df"]
info_id_df = data_id_gr["INFO_ID_df"]
network_id_df = data_id_gr["NETWORK_ID_df"]

# Instantiate object from mysql table creator class 
table_creator = MySQL_Table_Creator(dataset)

# Create dataset
visual_data = table_creator.combine(screen_id_df, visual_features)
hardware_data = table_creator.combine(hardware_id_df, hardware_features)
processor_data = table_creator.combine(processor_id_df, processor_features)
memory_data = table_creator.combine(memory_id_df, memory_features)
information_data = table_creator.combine(info_id_df, information_features)

## Data Storage

In [15]:
# Import functionalities for data storage
from scripts.collection.collector import DataSaver

In [16]:
# Instantiate datasaver 
datasever = DataSaver(folder="data\processed")
content_names = ["visual", "hardware", "processor", "memory", "information"]
filenames = [f"{name}_table" for name in content_names]

# Iteration: build a dictionaries of dataset for storage
sql_tables = [visual_data, hardware_data, processor_data, memory_data, information_data]
table_dict = {}
for table, filename in zip(sql_tables, filenames):
    # Store sql table for each filename
    table_dict[filename] = table

In [17]:
# Store all available tables into processed folder 
datasever.save_multiple_ds(table_dict)

File 1: dataset as visual_table.csv --> C:\Development\Projects\MachineLearning\Laptop-Price-Predictor-System\data\processed (Saving is successful
File 2: dataset as hardware_table.csv --> C:\Development\Projects\MachineLearning\Laptop-Price-Predictor-System\data\processed (Saving is successful
File 3: dataset as processor_table.csv --> C:\Development\Projects\MachineLearning\Laptop-Price-Predictor-System\data\processed (Saving is successful
File 4: dataset as memory_table.csv --> C:\Development\Projects\MachineLearning\Laptop-Price-Predictor-System\data\processed (Saving is successful
File 5: dataset as information_table.csv --> C:\Development\Projects\MachineLearning\Laptop-Price-Predictor-System\data\processed (Saving is successful
